# Infrustructure as Code

### Import libraries

In [ ]:
import boto3
import configparser
from botocore.exceptions import ClientError
import json
import psycopg2
%load_ext sql

### Read in configuration file and assign config data to variables

In [ ]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

key = config.get('AWS','key')
secret = config.get('AWS','secret')

cluster_identifier = config.get('REDSHIFT','cluster_identifier')
cluster_type = config.get('REDSHIFT','cluster_type')
node_type = config.get('REDSHIFT','node_type')
database_name = config.get('REDSHIFT','db_name')
username = config.get('REDSHIFT','username')
password = config.get('REDSHIFT','password')
port = config.get('REDSHIFT','port')

role_name = config.get('IAM', 'role_name')
policy_name = config.get('IAM', 'policy_name')

### Create an IAM client

In [ ]:
iam = boto3.client('iam',aws_access_key_id=key,
                     aws_secret_access_key=secret,
                     region_name='us-east-1'
                  )

### Create a role to attach to IAM 

In [ ]:
try:
    warehouse_role = iam.create_role(
        RoleName = role_name,
        Description = 'Allows Redshift cluster to call AWS services on behalf of the user',
        AssumeRolePolicyDocument = json.dumps(
            {
                'Statement': [
                    {
                        'Action': 'sts:AssumeRole',
                        'Effect': 'Allow',
                        'Principal': {
                            'Service': 'redshift.amazonaws.com'
                        }
                     }
                ],
                'Version': '2012-10-17'
            }
        )
    )
    print(f"Role named '{role_name}' has been created.")

except Exception as e:
    print(str(e))    

### Attach s3 access policy to IAM Role

In [ ]:
try:
    iam.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
    )['ResponseMetadata']['HTTPStatusCode']   
except Exception as e:
    print(str(e))

### Extract ARN from IAM Role

In [ ]:
role_arn = iam.get_role(
    RoleName = role_name
)['Role']['Arn']

print(f"Role {role_name}'s ARN is: {role_arn}.")

## Redshift

### Create a Redshift client 

In [ ]:
redshift = boto3.client('redshift',
                       region_name='us-east-1',
                       aws_access_key_id=key,
                       aws_secret_access_key=secret
                       )

### Check to see if a cluster already exists, delete if true

In [ ]:
try:
    redshift.delete_cluster(
        ClusterIdentifier = cluster_identifier,
        SkipFinalClusterSnapshot = True,
    )

    print(f"The cluster '{cluster_identifier}' already exists.")
    print(f"Deleting '{cluster_identifier}'...")
    
    # Create a waiter object that will check to see if cluster has been deleted if it exists 
    # Checks every 20 seconds, 30 tries if needed
    redshift_waiter = redshift.get_waiter('cluster_deleted')
    redshift_waiter.wait(
        ClusterIdentifier = cluster_identifier,
        WaiterConfig={
            'Delay': 20,
            'MaxAttempts':20
        }
    )
    print(f"{cluster_identifier} was successfully deleted.")
    
except:
    print(f"There are no clusters called {cluster_identifier}.")
    

### Create a new cluster

In [ ]:
try:
    cluster = redshift.create_cluster(
        DBName=database_name,
        ClusterIdentifier=cluster_identifier,
        ClusterType=cluster_type,
        NodeType=node_type,
        MasterUsername=username,
        MasterUserPassword=password,
        Port=int(port),
        IamRoles=[role_arn]
    )

    print(f"Creating a new cluster called {cluster_identifier}.")
    
    # Create a waiter object that will check to see if cluster is available
    # Checks every 20 seconds, 30 tries if needed
    waiter = redshift.get_waiter('cluster_available')
    waiter.wait(
        ClusterIdentifier=cluster_identifier,
        WaiterConfig={
                'Delay': 20,
                'MaxAttempts':20
            }
    )

    print(f"{cluster_identifier} is up and running.")

except Exception as e:
    print(str(e)) 

### Get cluster endpoint and vpc security group

In [ ]:
clusters = redshift.describe_clusters(
    ClusterIdentifier=cluster_identifier
)['Clusters']
        
cluster_endpoint = clusters[0]['Endpoint']['Address']
vpc_security_group_id = clusters[0]["VpcSecurityGroups"][0]['VpcSecurityGroupId']

print(f"The endpoint for {cluster_identifier} is {cluster_endpoint}.")
print(f"The VPC security group ID for '{vpc_security_group_id}' is '{cluster_identifier}'.")

## EC2

### Create an ec2 instance

In [ ]:
try:
    ec2 = boto3.resource('ec2',
                         region_name='us-east-1',
                         aws_access_key_id=key,
                         aws_secret_access_key=secret
    )

    ec2_security_group = ec2.SecurityGroup(vpc_security_group_id)

    ec2_security_group.authorize_ingress(
        GroupName = ec2_security_group.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(port),
        ToPort=int(port)
    )
except Exception as e:
    print(e)

## Create a connection to the database and test

In [ ]:
conn_string = f"postgresql://{username}:{password}@{cluster_endpoint}:{port}/{database_name}"

print(conn_string)

%sql $conn_string

## Delete Redshift cluster

In [ ]:
try:
    redshift.delete_cluster(
        ClusterIdentifier = cluster_identifier,
        SkipFinalClusterSnapshot = True,
    )
    
    print(f"Now deleting {cluster_identifier}...")
    
    # Create a waiter object that will check to see if cluster has been deleted
    # Checks every 20 seconds, 30 tries if needed
    delete_waiter = redshift.get_waiter('cluster_deleted')
    delete_waiter.wait(
        ClusterIdentifier=cluster_identifier,
        WaiterConfig={
            'Delay': 20,
            'MaxAttempts':30
        }
    )
    
    print(f"{cluster_identifier} has been deleted.")
    
except :
    print(f"{cluster_identifier} does not exist.")    

## Detach S3 policy from IAM role 

In [ ]:
try:
    iam.detach_role_policy(RoleName=role_name, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
    iam.delete_role(RoleName=role_name)
except :
    print(f"{role_name} does not exist.") 